In [1]:
!git clone https://github.com/Oliver-Lai/monopoly

Cloning into 'monopoly'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 17 (delta 4), reused 13 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 524.80 KiB | 6.10 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
import numpy as np
import random as rd
from monopoly.playground import monopoly_with_fair_prison

rd.seed(42)
NUM_SIMULATIONS = 2000
# 固定數值
GAME_MAP = [rd.randint(400,500) for i in range(10)]
PLAYER_NUM = 4
PRISON_LOCATION = 5
PRISON_DAY = 2

# 輸入變動數值
# PLAYERS_MONEY
# PRISON_FINE

In [3]:
input_data = np.array([[rd.randint(500,2500),rd.randint(35,150)] for i in range(3000)])
input_data

array([[1885,  129],
       [2327,  104],
       [ 678,  110],
       ...,
       [ 685,   71],
       [ 556,   84],
       [ 601,   55]])

In [4]:
output = []
for i in input_data:
    turn = []
    PLAYERS_MONEY = i[0]
    PRISON_FINE = i[1]
    for j in range(NUM_SIMULATIONS):
        game = monopoly_with_fair_prison(PLAYERS_MONEY,PLAYER_NUM,GAME_MAP,PRISON_LOCATION,PRISON_DAY,j,PRISON_FINE)
        game.play_game()
        turn.append(game.get_turn())
    output.append(round(np.mean(turn),2))

output

[43.44,
 81.67,
 19.41,
 26.27,
 22.6,
 20.12,
 27.17,
 17.17,
 17.39,
 41.37,
 34.16,
 18.19,
 44.52,
 55.07,
 18.53,
 93.62,
 56.42,
 21.89,
 22.6,
 78.26,
 23.21,
 21.92,
 22.25,
 86.55,
 46.83,
 107.47,
 60.12,
 39.06,
 106.98,
 19.04,
 17.07,
 40.71,
 64.32,
 20.44,
 17.11,
 25.69,
 59.14,
 76.89,
 15.76,
 15.93,
 22.01,
 18.99,
 37.28,
 22.98,
 21.9,
 17.16,
 17.39,
 79.64,
 65.34,
 38.27,
 18.04,
 71.6,
 19.43,
 22.54,
 80.77,
 50.08,
 16.86,
 17.45,
 53.12,
 22.88,
 112.05,
 79.26,
 23.34,
 23.2,
 83.22,
 69.56,
 21.53,
 47.14,
 19.19,
 78.57,
 28.36,
 21.52,
 16.57,
 34.0,
 17.09,
 39.33,
 79.72,
 22.43,
 20.99,
 28.94,
 17.45,
 15.13,
 25.38,
 55.68,
 59.82,
 21.84,
 19.96,
 20.73,
 25.02,
 18.2,
 127.51,
 44.25,
 18.36,
 80.56,
 74.75,
 58.11,
 23.06,
 19.64,
 20.11,
 19.32,
 69.05,
 18.36,
 82.92,
 34.91,
 72.21,
 17.29,
 45.0,
 49.88,
 21.06,
 83.75,
 28.27,
 86.98,
 142.57,
 44.7,
 34.33,
 21.58,
 64.8,
 80.64,
 20.52,
 15.81,
 46.72,
 17.19,
 23.57,
 16.28,
 27.7,
 24.0,

In [5]:
output = np.array(output)

In [6]:
output

array([43.44, 81.67, 19.41, ..., 23.18, 19.04, 24.1 ])

In [77]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
input_data_norm = normalize(input_data,norm='l1')
X_train, X_test, y_train, y_test = train_test_split(input_data,output ,
                                   random_state=104,
                                   test_size=0.3,
                                   shuffle=True)

In [78]:
X_train

array([[1429,   69],
       [2110,   99],
       [1675,   64],
       ...,
       [2410,  126],
       [1439,  117],
       [2372,  117]])

In [79]:
model = MLPRegressor(solver="adam",activation="relu",max_iter=20000,hidden_layer_sizes=[20,10,5])

In [80]:
model.fit(X_train,y_train)

MLPRegressor(hidden_layer_sizes=[20, 10, 5], max_iter=20000)

In [81]:
y_pred = model.predict(X_train)

In [82]:
from sklearn.metrics import root_mean_squared_error
root_mean_squared_error(y_train,y_pred)

8.446195671496579

In [83]:
y_pred = model.predict(X_test)

In [84]:
root_mean_squared_error(y_test,y_pred)

8.65030273658041

In [85]:
test = normalize([[3000,75]],norm='l1')
y_pred = model.predict([[3000,75]])
y_pred

array([125.51369501])